# Loading Set

In [13]:
from keras import losses, metrics
from keras.models import Sequential
from keras.models import Model, load_model
from keras.optimizers import Adam as Adam
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPool1D, BatchNormalization, Dropout, Activation
from keras.layers import GlobalAveragePooling1D, Flatten, SeparableConv1D, Input, LSTM, GRU, concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import auc, classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score, precision_recall_curve
import tensorflow as tf
import os, sys, pickle
import pandas as pd
import random

In [14]:
print('loading train...', flush=True, end='')

# x를 loading해서 (batch_size, step, channel)
input_path = '../dataset/preprocess4/welch/'
x_train = np.load(input_path+'x_train.npz', allow_pickle=True)['arr_0']
x_test = np.load(input_path+'x_test.npz', allow_pickle=True)['arr_0']
x_val = np.load(input_path+'x_val.npz', allow_pickle=True)['arr_0']
y_train = np.load(input_path+'y_train.npz')['arr_0']
y_test = np.load(input_path+'y_test.npz')['arr_0']
y_val = np.load(input_path+'y_val.npz')['arr_0']
print('done', flush=True)


x_train_ecg = x_train[:,:,:,1]
x_val_ecg = x_val[:,:,:,1]
x_test_ecg = x_test[:,:,:,1]


print('x_train shape:', x_train_ecg.shape)
print('x_test.shape:', x_test_ecg.shape)
print('x_val.shape:', x_val_ecg.shape)


loading train...done
x_train shape: (74626, 16, 128)
x_test.shape: (8792, 16, 128)
x_val.shape: (7952, 16, 128)


## Binary

In [15]:
# binary classification
y_train_bin = y_train >= 4
y_test_bin = y_test >= 4
y_val_bin = y_val >= 4
y_train_bin2 = y_train >=7
y_val_bin2 = y_val>=7
y_test_bin2 = y_test>=7

## 3 class

In [4]:
### 3 classes
# labels for y_train
y_train_class = []
for i in range(y_train.shape[0]):
    if y_train[i] <= 3.5:
        y_train_class.append((1,0,0))
    elif 3.5<y_train[i]<=6.5:
        y_train_class.append((0,1,0))
    else:
        y_train_class.append((0,0,1))
        
y_train_class = np.array(y_train_class, int)


# labels for y_val
y_val_class = []
for i in range(y_val.shape[0]):
    if y_val[i] <= 3.5:
        y_val_class.append((1,0,0))
    elif 3.5<y_val[i]<=6.5:
        y_val_class.append((0,1,0))
    else:
        y_val_class.append((0,0,1))
        
y_val_class = np.array(y_val_class, int)

# labels for y_test
y_test_class = []
for i in range(y_test.shape[0]):
    if y_test[i] <= 3.5:
        y_test_class.append([1,0,0])
    elif 3.5<y_test[i]<=6.5:
        y_test_class.append([0,1,0])
    else:
        y_test_class.append([0,0,1])
        
y_test_class = np.array(y_test_class, int)

## Sample weight

### 2 class (NRS>=4, NRS<4)

In [4]:
# 2 class에 대한 sample weight
train_w_samp2 = np.ones(shape=(len(y_train),))
train_w_samp2[y_train_bin==0]= len(y_train) / np.sum(y_train_bin)
train_w_samp2[y_train_bin!=0]= len(y_train) / np.sum(~y_train_bin)

train_w_samp2_2 = np.ones(shape=(len(y_train),))
train_w_samp2_2[y_train_bin==0]= len(y_train) / np.sum(y_train_bin2)
train_w_samp2_2[y_train_bin!=0]= len(y_train) / np.sum(~y_train_bin2)

print('sample weight for no pain: {:.2f}, moderate pain: {:.2f}'
      .format(len(y_train) / np.sum(y_train_bin), len(y_train) / np.sum(~y_train_bin)))
print('sample weight for no pain: {:.2f}, severe pain: {:.2f}'
      .format(len(y_train) / np.sum(y_train_bin2), len(y_train) / np.sum(~y_train_bin2)))


# 2 class에 대한 sample weight
val_w_samp2 = np.ones(shape=(len(y_val),))
val_w_samp2[y_val_bin==0]= len(y_val) / np.sum(y_val_bin)
val_w_samp2[y_val_bin!=0]= len(y_val) / np.sum(~y_val_bin)

val_w_samp2_2 = np.ones(shape=(len(y_val),))
val_w_samp2_2[y_val_bin==0]= len(y_val) / np.sum(y_val_bin2)
val_w_samp2_2[y_val_bin!=0]= len(y_val) / np.sum(~y_val_bin2)

print('sample weight for no pain: {:.2f}, moderate pain: {:.2f}'
      .format(len(y_val) / np.sum(y_val_bin), len(y_val) / np.sum(~y_val_bin)))
print('sample weight for no pain: {:.2f}, severe pain: {:.2f}'
      .format(len(y_val) / np.sum(y_val_bin2), len(y_val) / np.sum(~y_val_bin2)))


# 2 class에 대한 sample weight
test_w_samp2 = np.ones(shape=(len(y_test),))
test_w_samp2[y_test_bin==0]= len(y_test) / np.sum(y_test_bin)
test_w_samp2[y_test_bin!=0]= len(y_test) / np.sum(~y_test_bin)

test_w_samp2_2 = np.ones(shape=(len(y_test),))
test_w_samp2_2[y_test_bin==0]= len(y_test) / np.sum(y_test_bin2)
test_w_samp2_2[y_test_bin!=0]= len(y_test) / np.sum(~y_test_bin2)

print('sample weight for no pain: {:.2f}, moderate pain: {:.2f}'
      .format(len(y_test) / np.sum(y_test_bin), len(y_test) / np.sum(~y_test_bin)))
print('sample weight for no pain: {:.2f}, severe pain: {:.2f}'
      .format(len(y_test) / np.sum(y_test_bin2), len(y_test) / np.sum(~y_test_bin2)))

sample weight for no pain: 1.62, moderate pain: 2.60
sample weight for no pain: 6.39, severe pain: 1.19
sample weight for no pain: 1.60, moderate pain: 2.68
sample weight for no pain: 6.74, severe pain: 1.17
sample weight for no pain: 1.71, moderate pain: 2.41
sample weight for no pain: 6.67, severe pain: 1.18


### 3 class (NRS>=7, 7>NRS>=4, NRS<4)

In [16]:
# 3 class에 대한 sample weight
train_w_samp3 = np.ones(shape=(len(y_train),))

train_w_samp3[y_train<4]= len(y_train)/np.sum(y_train<4)
train_w_samp3[(y_train>=4)&(y_train<7)]= len(y_train)/np.sum((4<=y_train)&(y_train<7))
train_w_samp3[y_train>=7]= len(y_train)/np.sum(y_train>=7)

print('train set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}\n'
      .format(len(y_train)/np.sum(y_train<4),len(y_train)/np.sum((y_train>=4)&(y_train<7)),len(y_train)/np.sum(y_train>=7)))

# 3 class에 대한 sample weight
val_w_samp3 = np.ones(shape=(len(y_val),))

val_w_samp3[y_val<4]= len(y_val)/np.sum(y_val<4)
val_w_samp3[(y_val>=4)&(y_val<7)]= len(y_val)/np.sum((4<=y_val)&(y_val<7))
val_w_samp3[y_val>=7]= len(y_val)/np.sum(y_val>=7)

print('val set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}\n'
      .format(len(y_val)/np.sum(y_val<4),len(y_val)/np.sum((y_val>=4)&(y_val<7)),len(y_val)/np.sum(y_val>=7)))

# 3 class에 대한 sample weight
test_w_samp3 = np.ones(shape=(len(y_test),))

test_w_samp3[y_test<4]= len(y_test)/np.sum(y_test<4)
test_w_samp3[(y_test>=4)&(y_test<7)]= len(y_test)/np.sum((4<=y_test)&(y_test<7))
test_w_samp3[y_test>=7]= len(y_test)/np.sum(y_test>=7)

print('test set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}'
      .format(len(y_test)/np.sum(y_test<4),len(y_test)/np.sum((y_test>=4)&(y_test<7)),len(y_test)/np.sum(y_test>=7)))

train set
sample weight for class 1: 2.84, class 2: 2.07, class 3: 6.08

val set
sample weight for class 1: 2.69, class 2: 2.09, class 3: 6.64

test set
sample weight for class 1: 2.62, class 2: 2.17, class 3: 6.37


# Settings

In [8]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

4 Physical GPUs, 4 Logical GPUs


In [17]:
# folder
nfold = 1  # 각각의 hyperparameter에 대해 k-fold 를 시행하고 평균을 구한다.
ntest = 100
rootdir = "NRS>=4_welch/LSTM_3layers_sample-weighted_3class_age+gender"

predirs = []
for root, dirs, files in os.walk(rootdir):  # 하위 대상들을 recursive 하게 긁어옴
    for filename in dirs:
        predirs.append(filename)

if not os.path.exists(rootdir):
    os.mkdir(rootdir)


# test_settings
test_settings_1, test_settings_2, test_settings_3 = [], [], []


# hyperparamters
#num_nodes = [64, 64, 64] #, 64, 64, 64]
#kernel_size = 10
pool_size = 2

#dense_node = 32
#dropout_rate = 0.2
learning_rate = 0.002

# hyperparamters pool
dropout_opts  = [0, 0.1, 0.2, 0.3, 0.4, 0.5] # dropout rate
dense_opts = [8, 16, 32, 64]
BATCH_SIZE = [512, 1024]


unit_opts = [4, 8, 16, 32, 64, 128, 256]

print('start making test settings...', end='', flush=True)
# test settings
for unit1 in unit_opts:
    for unit2 in unit_opts:
        for unit3 in unit_opts:
            for dense_node1 in dense_opts:
                for dense_node2 in dense_opts:
                    for dropout1 in dropout_opts:
                        for dropout2 in dropout_opts:
                            for dropout3 in dropout_opts:
                                for batch_size in BATCH_SIZE:
                                    test_settings_1.append([unit1, unit2, unit3, dense_node1, dense_node2, dropout1, dropout2, dropout3, batch_size])

print('done')

start making test settings...done


# RandomSearch

## binary

In [ ]:
from keras import metrics
# random search for hyperparameter
ntrial = 400
train_errs, val_errs = [] ,[]
test_roc, test_prc = [], []
test_acc = []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
        
        
    rnn_type = random.choice(['lstm','gru'])
        
    # test_setting
    unit1, unit2, unit3, dense_node1, dense_node2, dropout1, dropout2, dropout3, batch_size = test_setting_1
    
    
    # total LSTM layers of the model
    n_unit = random.choice([1,2,3])   

    if n_unit==1:
        units = [unit1]

    if n_unit==2:
        units = [unit1, unit2]
        
    if n_unit==3:
        units = [unit1, unit2, unit3]
        
        
    # final dense layers
    n_dense = random.choice([0,1,2])
    
    if n_dense ==0:
        dense_node1 = 0
        dropout2 = 0 
        dense_node2 = 0
        dropout3 = 0
    
    if n_dense ==1:
        dense_node2 = 0
        dropout3 = 0
        
    
    
    # 이번 옵션에 대한 결과 디렉토리
    odir_f = '{}_batch={},'.format(rnn_type, batch_size)    
    for unit in units:
        odir_f += 'unit{},'.format(unit)
    odir_f += 'dropout={},dnodes={},dropout={},dnodes={},dropout={}'.format(dropout1, dense_node1, dropout2, dense_node2, dropout3)
    random_settings.append(odir_f)
    
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/weights.hdf5".format(odir)        

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    with strategy.scope():
        
        # build a model - function api
        inp = Input(shape=(x_train.shape[1], x_train_ecg.shape[2]))

        if rnn_type == 'lstm':
            # lstm layer
            inp_lstm = inp
            for unit in units[:-1]:
                inp_lstm = LSTM(unit, return_sequences=True) (inp_lstm)
            out_lstm = LSTM(units[-1]) (inp_lstm)

        else:
            # gru layer
            inp_lstm = inp
            for unit in units[:-1]:
                inp_lstm = GRU(unit, return_sequences=True) (inp_lstm)
            out_lstm = GRU(units[-1]) (inp_lstm)            
            
            
        out_lstm = Dropout(dropout1) (out_lstm)

        if n_dense >= 1:
            out_lstm = Dense(dense_node1, activation='tanh') (out_lstm)
            out_lstm = Dropout(dropout2) (out_lstm)
            
        if n_dense >= 2:
            out_lstm = Dense(dense_node2, activation='tanh') (out_lstm)
            out_lstm = Dropout(dropout3) (out_lstm)   

        # output
        out = Dense(1, activation='sigmoid') (out_lstm)
        model = Model(inputs=[inp], outputs=[out])
        



        # model 학습 설정
        try:
            model.compile(loss='binary_crossentropy', optimizer=Adam(lr=learning_rate), metrics=["acc", tf.keras.metrics.AUC()])
            hist = model.fit(x_train_ecg, y_train_bin, sample_weight=train_w_samp3, validation_data=(x_val_ecg, y_val_bin, val_w_samp3), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')])
        except:
            os.rmdir(odir)
            #os.rename(odir,rootdir+'/error_{}'.format(odir_f))
            itrial -= 1
            test_roc.append(0)
            test_acc.append(0)
            test_prc.append(0)
            train_errs.append(-1)
            val_errs.append(-1)
            continue
            
            
    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_test_ecg).flatten()

    
    # acc 계산
    acc = metrics.Accuracy()
    acc.update_state(y_pred>=0.5, y_test_bin, sample_weight=test_w_samp3)
    acc_val = acc.result().numpy()
    test_acc.append(acc_val)
    
    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test_bin, y_pred, sample_weight=test_w_samp3)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_roc.append(roc_auc)

    # auprc 
    precision, recall, _ = precision_recall_curve(y_test_bin, y_pred, sample_weight=test_w_samp3)
    prc_auc = auc(recall, precision)
    test_prc.append(prc_auc)

    
    # rename
    os.rename(odir, rootdir+'/roc{:.4f}_prc{:.4f}_{}_acc{:.2f}'.format(roc_auc, prc_auc, odir_f, acc_val))

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_roc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_roc(max_idx), random_settings(max_idx)))


random search 0/400
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Epoch 1/100
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcas

## age+gender

In [18]:
import pandas as pd
import pickle

print('loading train...', flush=True, end='')

# x를 loading해서 (batch_size, step, channel)
input_path = '../dataset/preprocess4/input2/'

gender_train = np.load(input_path+'gender_train.npz', allow_pickle=True)['arr_0']
gender_test = np.load(input_path+'gender_test.npz', allow_pickle=True)['arr_0']
gender_val = np.load(input_path+'gender_val.npz', allow_pickle=True)['arr_0']

age_train = np.load(input_path+'age_train.npz', allow_pickle=True)['arr_0']
age_test = np.load(input_path+'age_test.npz', allow_pickle=True)['arr_0']
age_val = np.load(input_path+'age_val.npz', allow_pickle=True)['arr_0']
print('done', flush=True)


input_path = '../dataset/preprocess4/welch/'
train_mask = pickle.load(open(input_path+'train_mask_pacu', 'rb')) + pickle.load(open(input_path+'train_mask_preop', 'rb'))
val_mask = pickle.load(open(input_path+'val_mask_pacu', 'rb')) + pickle.load(open(input_path+'val_mask_preop', 'rb'))
test_mask = pickle.load(open(input_path+'test_mask_pacu', 'rb')) + pickle.load(open(input_path+'test_mask_preop', 'rb'))

print(len(train_mask), np.sum(train_mask), len(x_train_ecg))

loading train...done
74636 74626 74626


In [21]:
len(x_train_ecg), len(gender_train), (np.sum(train_mask))

(74626, 74636, 74626)

In [22]:
# input, sample weight settings
train_w_samp = train_w_samp3
val_w_samp = val_w_samp3
test_w_samp = test_w_samp3

# 
gender_train = gender_train[train_mask]
gender_val = gender_val[val_mask]
gender_test = gender_test[test_mask]

age_train = age_train[train_mask]
age_val = age_val[val_mask]
age_test = age_test[test_mask]


agender_train = np.array([[age_train[i], gender_train[i]] for i in range(len(age_train))])
agender_val = np.array([[age_val[i], gender_val[i]] for i in range(len(age_val))])
agender_test = np.array([[age_test[i], gender_test[i]] for i in range(len(age_test))])

x_trains = [x_train_ecg, agender_train]
x_vals = [x_val_ecg, agender_val]
x_tests = [x_test_ecg, agender_test]

In [23]:
from keras import metrics
# random search for hyperparameter
ntrial = 700
train_errs, val_errs = [] ,[]
test_roc, test_prc = [], []
test_acc = []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
    
    rnn_type = random.choice(['lstm','gru'])
        
    # test_setting
    unit1, unit2, unit3, dense_node1, dense_node2, dropout1, dropout2, dropout3, batch_size = test_setting_1
    
    
    # total LSTM layers of the model
    n_unit = random.choice([1,2,3])   

    if n_unit==1:
        units = [unit1]

    if n_unit==2:
        units = [unit1, unit2]
        
    if n_unit==3:
        units = [unit1, unit2, unit3]
        
        
    # final dense layers
    n_dense = random.choice([0,1])
    
    if n_dense ==0:
        dense_node1 = 0
        dropout2 = 0 
        dense_node2 = 0
        dropout3 = 0
    
    if n_dense ==1:
        dense_node2 = 0
        dropout3 = 0
        
    
    
    # 이번 옵션에 대한 결과 디렉토리
    odir_f = '{}_batch={},'.format(rnn_type, batch_size)    
    for unit in units:
        odir_f += 'unit{},'.format(unit)
    odir_f += 'dropout={},dnodes={},dropout={}'.format(dropout1, dense_node1, dropout2)
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/weights.hdf5".format(odir)        

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:1"])
    with strategy.scope():
        
        # build a model - function api
        inp_fnn = Input(shape=(agender_train.shape[1],))
        inp = Input(shape=(x_train_ecg.shape[1], x_train_ecg.shape[2]))
        inp_lstm = inp
        
        out_fnn = inp_fnn
        out_fnn = Activation('sigmoid') (out_fnn)
        

        if rnn_type == 'lstm':
            # lstm layer
            for unit in units[:-1]:
                inp_lstm = LSTM(unit, return_sequences=True) (inp_lstm)
            out_lstm = LSTM(units[-1]) (inp_lstm)

        else:
            # gru layer
            for unit in units[:-1]:
                inp_lstm = GRU(unit, return_sequences=True) (inp_lstm)
            out_lstm = GRU(units[-1]) (inp_lstm)            
            
            
        out_lstm = Dropout(dropout1) (out_lstm)
        out = concatenate([out_fnn, out_lstm])

        if n_dense >= 1:
            out = Dense(dense_node1, activation='tanh') (out)
            out = Dropout(dropout2) (out)         
            

        # output
        out = Dense(1, activation='sigmoid') (out)
        model = Model(inputs=[inp, inp_fnn], outputs=[out])


        # model 학습 설정
        try:
            model.compile(loss='binary_crossentropy', optimizer=Adam(lr=learning_rate), metrics=["acc", tf.keras.metrics.AUC()])
            hist = model.fit(x_trains, y_train_bin, sample_weight=train_w_samp3, validation_data=(x_vals, y_val_bin, val_w_samp3), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])
        except Exception as e:
            print(e)
            os.rmdir(odir)
            #os.rename(odir,rootdir+'/error_{}'.format(odir_f))
            itrial -= 1
            test_roc.append(0)
            test_acc.append(0)
            test_prc.append(0)
            train_errs.append(-1)
            val_errs.append(-1)
            continue
            
            
    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_tests).flatten()

    
    # acc 계산
    acc = metrics.Accuracy()
    acc.update_state(y_pred>=0.5, y_test_bin, sample_weight=test_w_samp3)
    acc_val = acc.result().numpy()
    test_acc.append(acc_val)
    
    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test_bin, y_pred, sample_weight=test_w_samp3)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_roc.append(roc_auc)

    # auprc 
    precision, recall, _ = precision_recall_curve(y_test_bin, y_pred, sample_weight=test_w_samp3)
    prc_auc = auc(recall, precision)
    test_prc.append(prc_auc)

    
    # rename
    os.rename(odir, rootdir+'/roc{:.4f}_prc{:.4f}_{}_acc{:.2f}'.format(roc_auc, prc_auc, odir_f, acc_val))

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_roc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_roc(max_idx), random_settings(max_idx)))


random search 0/700
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:1',)
Epoch 1/100
73/73 [==============================] - ETA: 0s - loss: 1.8934 - acc: 0.6448 - auc_1: 0.5633
Epoch 00001: val_loss improved from inf to 1.91956, saving model to NRS>=4_welch/LSTM_3layers_sample-weighted_3class_age+gender/gru_batch=1024,unit16,unit4,unit128,dropout=0.2,dnodes=64,dropout=0/weights.hdf5
73/73 [==============================] - 11s 153ms/step - loss: 1.8934 - acc: 0.6448 - auc_1: 0.5633 - val_loss: 1.9196 - val_acc: 0.6221 - val_auc_1: 0.5759
Epoch 2/100
73/73 [==============================] - ETA: 0s - loss: 1.8346 - acc: 0.6631 - auc_1: 0.6237
Epoch 00002: val_loss did not improve from 1.91956
73/73 [==============================] - 8s 112ms/step - loss: 1.8346 - acc: 0.6631 - auc_1: 0.6237 - val_loss: 1.9277 - val_acc: 0.6275 - val_auc_1: 0.5804
Epoch 3/100
73/73 [==============================] - ETA: 0s - loss: 1.7969 - acc: 0.6763 -

71/73 [============================>.] - ETA: 0s - loss: 1.8488 - acc: 0.6607 - auc: 0.6084
Epoch 00010: val_loss did not improve from 1.89555
73/73 [==============================] - 1s 13ms/step - loss: 1.8481 - acc: 0.6610 - auc: 0.6090 - val_loss: 1.8999 - val_acc: 0.6225 - val_auc: 0.5734
random search 2/700
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:1',)
Epoch 1/100
143/146 [============================>.] - ETA: 0s - loss: 1.8656 - acc: 0.6518 - auc: 0.5984
Epoch 00001: val_loss improved from inf to 1.91404, saving model to NRS>=4_welch/LSTM_3layers_sample-weighted_3class_age+gender/lstm_batch=512,unit32,unit4,dropout=0,dnodes=0,dropout=0/weights.hdf5
146/146 [==============================] - 6s 39ms/step - loss: 1.8657 - acc: 0.6521 - auc: 0.5992 - val_loss: 1.9140 - val_acc: 0.6317 - val_auc: 0.5697
Epoch 2/100
145/146 [============================>.] - ETA: 0s - loss: 1.8034 - acc: 0.6766 - auc: 0.6475
Epoch 00002: val_lo

Epoch 1/100
73/73 [==============================] - ETA: 0s - loss: 1.8858 - acc: 0.6503 - auc: 0.5953
Epoch 00001: val_loss improved from inf to 1.89580, saving model to NRS>=4_welch/LSTM_3layers_sample-weighted_3class_age+gender/gru_batch=1024,unit256,unit256,unit4,dropout=0.2,dnodes=8,dropout=0.4/weights.hdf5
73/73 [==============================] - 11s 154ms/step - loss: 1.8858 - acc: 0.6503 - auc: 0.5953 - val_loss: 1.8958 - val_acc: 0.6421 - val_auc: 0.5770
Epoch 2/100
73/73 [==============================] - ETA: 0s - loss: 1.7973 - acc: 0.6880 - auc: 0.6550
Epoch 00002: val_loss did not improve from 1.89580
73/73 [==============================] - 8s 105ms/step - loss: 1.7973 - acc: 0.6880 - auc: 0.6550 - val_loss: 1.8986 - val_acc: 0.6557 - val_auc: 0.5741
Epoch 3/100
73/73 [==============================] - ETA: 0s - loss: 1.7780 - acc: 0.6934 - auc: 0.6634
Epoch 00003: val_loss did not improve from 1.89580
73/73 [==============================] - 8s 105ms/step - loss: 1.778

Epoch 18/100
73/73 [==============================] - ETA: 0s - loss: 1.8021 - acc: 0.6770 - auc: 0.6489
Epoch 00018: val_loss did not improve from 1.90265
73/73 [==============================] - 3s 44ms/step - loss: 1.8021 - acc: 0.6770 - auc: 0.6489 - val_loss: 1.9240 - val_acc: 0.6374 - val_auc: 0.5763
Epoch 19/100
73/73 [==============================] - ETA: 0s - loss: 1.7949 - acc: 0.6808 - auc: 0.6505
Epoch 00019: val_loss improved from 1.90265 to 1.89505, saving model to NRS>=4_welch/LSTM_3layers_sample-weighted_3class_age+gender/gru_batch=1024,unit16,dropout=0.1,dnodes=0,dropout=0/weights.hdf5
73/73 [==============================] - 3s 45ms/step - loss: 1.7949 - acc: 0.6808 - auc: 0.6505 - val_loss: 1.8951 - val_acc: 0.6397 - val_auc: 0.5842
Epoch 20/100
73/73 [==============================] - ETA: 0s - loss: 1.7902 - acc: 0.6826 - auc: 0.6528
Epoch 00020: val_loss did not improve from 1.89505
73/73 [==============================] - 3s 44ms/step - loss: 1.7902 - acc: 0.682

70/73 [===========================>..] - ETA: 0s - loss: 1.8734 - acc: 0.6519 - auc: 0.5874
Epoch 00001: val_loss improved from inf to 1.91749, saving model to NRS>=4_welch/LSTM_3layers_sample-weighted_3class_age+gender/lstm_batch=1024,unit16,unit16,dropout=0.1,dnodes=0,dropout=0/weights.hdf5
73/73 [==============================] - 5s 65ms/step - loss: 1.8712 - acc: 0.6524 - auc: 0.5905 - val_loss: 1.9175 - val_acc: 0.6230 - val_auc: 0.5662
Epoch 2/100
73/73 [==============================] - ETA: 0s - loss: 1.8112 - acc: 0.6713 - auc: 0.6426
Epoch 00002: val_loss did not improve from 1.91749
73/73 [==============================] - 1s 16ms/step - loss: 1.8112 - acc: 0.6713 - auc: 0.6426 - val_loss: 1.9374 - val_acc: 0.6265 - val_auc: 0.5573
Epoch 3/100
73/73 [==============================] - ETA: 0s - loss: 1.7736 - acc: 0.6873 - auc: 0.6649
Epoch 00003: val_loss did not improve from 1.91749
73/73 [==============================] - 1s 16ms/step - loss: 1.7736 - acc: 0.6873 - auc: 0.

...

Epoch 1/100
73/73 [==============================] - ETA: 0s - loss: 1.9775 - acc: 0.6210 - auc: 0.5222
Epoch 00001: val_loss improved from inf to 1.90692, saving model to NRS>=4_welch/LSTM_3layers_sample-weighted_3class_age+gender/lstm_batch=1024,unit32,dropout=0,dnodes=16,dropout=0.5/weights.hdf5
73/73 [==============================] - 4s 55ms/step - loss: 1.9775 - acc: 0.6210 - auc: 0.5222 - val_loss: 1.9069 - val_acc: 0.6286 - val_auc: 0.5462
Epoch 2/100
71/73 [============================>.] - ETA: 0s - loss: 1.9050 - acc: 0.6440 - auc: 0.5617
Epoch 00002: val_loss did not improve from 1.90692
73/73 [==============================] - 1s 13ms/step - loss: 1.9048 - acc: 0.6442 - auc: 0.5618 - val_loss: 1.9124 - val_acc: 0.6207 - val_auc: 0.5521
Epoch 3/100
71/73 [============================>.] - ETA: 0s - loss: 1.8747 - acc: 0.6543 - auc: 0.5896
Epoch 00003: val_loss improved from 1.90692 to 1.90519, saving model to NRS>=4_welch/LSTM_3layers_sample-weighted_3class_age+gender/lstm_

Epoch 8/100
142/146 [============================>.] - ETA: 0s - loss: 1.8797 - acc: 0.6473 - auc: 0.5835
Epoch 00008: val_loss did not improve from 1.88832
146/146 [==============================] - 2s 11ms/step - loss: 1.8799 - acc: 0.6474 - auc: 0.5833 - val_loss: 1.8977 - val_acc: 0.6286 - val_auc: 0.5563
Epoch 9/100
144/146 [============================>.] - ETA: 0s - loss: 1.8786 - acc: 0.6471 - auc: 0.5839
Epoch 00009: val_loss did not improve from 1.88832
146/146 [==============================] - 2s 11ms/step - loss: 1.8787 - acc: 0.6473 - auc: 0.5838 - val_loss: 1.8966 - val_acc: 0.6286 - val_auc: 0.5529
Epoch 10/100
142/146 [============================>.] - ETA: 0s - loss: 1.8756 - acc: 0.6474 - auc: 0.5896
Epoch 00010: val_loss did not improve from 1.88832
146/146 [==============================] - 2s 11ms/step - loss: 1.8763 - acc: 0.6474 - auc: 0.5896 - val_loss: 1.8894 - val_acc: 0.6286 - val_auc: 0.5660
random search 693/700
INFO:tensorflow:Using MirroredStrategy with 

143/146 [============================>.] - ETA: 0s - loss: 1.8480 - acc: 0.6571 - auc: 0.6151
Epoch 00001: val_loss improved from inf to 1.91011, saving model to NRS>=4_welch/LSTM_3layers_sample-weighted_3class_age+gender/lstm_batch=512,unit32,unit16,unit8,dropout=0.2,dnodes=16,dropout=0/weights.hdf5
146/146 [==============================] - 6s 43ms/step - loss: 1.8471 - acc: 0.6574 - auc: 0.6155 - val_loss: 1.9101 - val_acc: 0.6319 - val_auc: 0.5795
Epoch 2/100
145/146 [============================>.] - ETA: 0s - loss: 1.7779 - acc: 0.6824 - auc: 0.6642
Epoch 00002: val_loss did not improve from 1.91011
146/146 [==============================] - 3s 18ms/step - loss: 1.7788 - acc: 0.6821 - auc: 0.6640 - val_loss: 1.9487 - val_acc: 0.6351 - val_auc: 0.5775
Epoch 3/100
146/146 [==============================] - ETA: 0s - loss: 1.7484 - acc: 0.6977 - auc: 0.6748
Epoch 00003: val_loss improved from 1.91011 to 1.89662, saving model to NRS>=4_welch/LSTM_3layers_sample-weighted_3class_age+ge

Epoch 4/100
73/73 [==============================] - ETA: 0s - loss: 1.7324 - acc: 0.7037 - auc: 0.6816
Epoch 00004: val_loss did not improve from 1.91527
73/73 [==============================] - 5s 74ms/step - loss: 1.7324 - acc: 0.7037 - auc: 0.6816 - val_loss: 1.9256 - val_acc: 0.6478 - val_auc: 0.5835
Epoch 5/100
73/73 [==============================] - ETA: 0s - loss: 1.7174 - acc: 0.7077 - auc: 0.6925
Epoch 00005: val_loss improved from 1.91527 to 1.91089, saving model to NRS>=4_welch/LSTM_3layers_sample-weighted_3class_age+gender/gru_batch=1024,unit128,unit32,dropout=0.4,dnodes=64,dropout=0.5/weights.hdf5
73/73 [==============================] - 6s 77ms/step - loss: 1.7174 - acc: 0.7077 - auc: 0.6925 - val_loss: 1.9109 - val_acc: 0.6527 - val_auc: 0.5920
Epoch 6/100
73/73 [==============================] - ETA: 0s - loss: 1.7104 - acc: 0.7087 - auc: 0.6955
Epoch 00006: val_loss did not improve from 1.91089
73/73 [==============================] - 6s 79ms/step - loss: 1.7104 - ac

NameError: name 'test_auc' is not defined

## 3 class

In [ ]:
from keras import metrics
from keras.layers import LeakyReLU, ReLU
from sklearn.metrics import roc_curve, auc, precision_recall_curve

# random search for hyperparameter
ntrial = 200
train_errs, val_errs = [] ,[]
test_roc, test_prc = [], []
test_rmse, test_acc = [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
    test_setting_2 = random.choice(test_settings_2)
    

    # test_setting
    num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4 = test_setting_1
    dense_node, dropout_cnn, dropout_fc, globalpool_opt, batch_size, conv_double = test_setting_2

        

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = 'batch={}, c1={}, c2={}, c3={}, c4={}, c1filts={}, c2filts={}, c3filts={}, c4filts={}, conv_double={}, globalpool_opt={}, dropout={}, dnodes={}, dropout={}'.format(batch_size, num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4, conv_double, globalpool_opt, dropout_cnn, dense_node, dropout_fc)
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/model.hdf5".format(odir)        

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2", "/gpu:3"])
    with strategy.scope():
        # build a model
        model = Sequential()

        conv_act = True
        if conv_act:
            act = 'relu'
        else:
            act = None

        # c1 layer
        if conv_double:
            model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='same'))
        model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))

        # c2 layer
        if conv_double:
            model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='same'))
        model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c3 layer
        if conv_double:
            model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='same'))
        model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c4 layer
        if conv_double:
            model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, padding='same'))
        model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))


        # global이냐 flatten이냐는 따로 모델 나눠야 할듯
        if globalpool_opt == 'max':
            model.add(GlobalMaxPool1D())
        elif globalpool_opt == 'ave':
            model.add(GlobalAveragePooling1D())
            
            
        if dense_node != 0:
            model.add(Dropout(dropout_cnn))
            model.add(Dense(dense_node, activation='tanh'))
        model.add(Dropout(dropout_fc))
        model.add(Dense(3, activation='softmax'))

        try:
            # model 학습 설정
            model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate), metrics=["acc", tf.keras.metrics.AUC()])
            hist = model.fit(x_train, y_train_class, validation_data=(x_val, y_val_class), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')])
        except:
            os.rename(odir,rootdir+'/error_{}'.format(odir_f))
            test_roc.append(0)
            test_acc.append(0)
            test_prc.append(0)
            train_errs.append(-1)
            val_errs.append(-1)
            continue

    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_test)

    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve((y_test_class[:,1]+y_test_class[:,2])>=1, y_pred[:,1]+y_pred[:,2])
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_roc.append(roc_auc)
    
    # auprc 
    precision, recall, _ = precision_recall_curve((y_test_class[:,1]+y_test_class[:,2])>=1, y_pred[:,1]+y_pred[:,2])
    prc_auc = auc(recall, precision)
    test_prc.append(prc_auc)
    
    # acc 계산
    l_test = np.argmax(y_test_class, axis=1)
    l_pred = np.argmax(y_pred, axis=1)
    acc_val = accuracy_score(l_test, l_pred)
    test_acc.append(acc_val)
    
    # rename
    os.rename(odir, rootdir+'/auc{:.4f}_prc{:.4f}_{}_acc{:.2f}'.format(roc_auc, prc_auc, odir_f, acc_val))

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_auc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_auc(max_idx), random_settings(max_idx)))

random search 0/200
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Epoch 1/100
307/307 [==============================] - ETA: 0s - loss: 1.0181 - acc: 0.4618 - auc_1: 0.6625
Epoch 00001: val_loss improved from inf to 0.98419, saving model to 3class/1D_CNN_4layers_outputs 1-fold test_setting 100 test data/batch=256, c1=256, c2=128, c3=32, c4=256, c1filts=3, c2filts=7, c3filts=5, c4filts=15, conv_double=True, globalpool_opt=max, dropout=0.3, dnodes=64, dropout=0/model.hdf5
307/307 [==============================] - 64s 209ms/step - loss: 1.0181 - acc: 0.4618 - auc_1: 0.6625 - val_loss: 0.9842 - val_acc: 0.4975 - val_auc_1: 0.6932
Epoch 2/100
307/307 [==============================] - ETA: 0s - loss: 1.0013 - acc: 0.4801 - auc_1: 0.6780
Epoch 00002: val_loss did not improve from 0.98419
307/307 [==============================] - 64s 207ms/step - loss: 1.0013 - acc: 0.4801 - auc_1: 0.6780

154/154 [==============================] - ETA: 0s - loss: 1.0134 - acc: 0.4582 - auc: 0.6586
Epoch 00001: val_loss improved from inf to 1.00113, saving model to 3class/1D_CNN_4layers_outputs 1-fold test_setting 100 test data/batch=512, c1=32, c2=512, c3=64, c4=128, c1filts=3, c2filts=15, c3filts=15, c4filts=3, conv_double=False, globalpool_opt=max, dropout=0.3, dnodes=8, dropout=0/model.hdf5
154/154 [==============================] - 35s 230ms/step - loss: 1.0134 - acc: 0.4582 - auc: 0.6586 - val_loss: 1.0011 - val_acc: 0.4792 - val_auc: 0.6784
Epoch 2/100
154/154 [==============================] - ETA: 0s - loss: 1.0079 - acc: 0.4648 - auc: 0.6655
Epoch 00002: val_loss improved from 1.00113 to 0.99208, saving model to 3class/1D_CNN_4layers_outputs 1-fold test_setting 100 test data/batch=512, c1=32, c2=512, c3=64, c4=128, c1filts=3, c2filts=15, c3filts=15, c4filts=3, conv_double=False, globalpool_opt=max, dropout=0.3, dnodes=8, dropout=0/model.hdf5
154/154 [===========================

Epoch 13/100
154/154 [==============================] - ETA: 0s - loss: 0.7435 - acc: 0.6760 - auc: 0.8432
Epoch 00013: val_loss did not improve from 0.85304
154/154 [==============================] - 20s 128ms/step - loss: 0.7435 - acc: 0.6760 - auc: 0.8432 - val_loss: 1.0475 - val_acc: 0.5725 - val_auc: 0.7498
random search 4/200
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Epoch 1/100
307/307 [==============================] - ETA: 0s - loss: 1.0651 - acc: 0.4326 - auc: 0.6361
Epoch 00001: val_loss improved from inf to 0.98852, saving model to 3class/1D_CNN_4layers_outputs 1-fold test_setting 100 test data/batch=256, c1=128, c2=64, c3=64, c4=128, c1filts=9, c2filts=13, c3filts=15, c4filts=15, conv_double=True, globalpool_opt=max, dropout=0.2, dnodes=8, dropout=0.5/model.hdf5
307/307 [==============================] - 38s 124ms/step - loss: 1.0651 - acc: 0.4326 - auc: 0.6361 - val_

307/307 [==============================] - 177s 577ms/step - loss: 0.8483 - acc: 0.6158 - auc: 0.7926 - val_loss: 0.8540 - val_acc: 0.6317 - val_auc: 0.7897
Epoch 8/100
307/307 [==============================] - ETA: 0s - loss: 0.8203 - acc: 0.6289 - auc: 0.8077
Epoch 00008: val_loss did not improve from 0.85400
307/307 [==============================] - 177s 575ms/step - loss: 0.8203 - acc: 0.6289 - auc: 0.8077 - val_loss: 0.8853 - val_acc: 0.6112 - val_auc: 0.7734
Epoch 9/100
307/307 [==============================] - ETA: 0s - loss: 0.7849 - acc: 0.6457 - auc: 0.8255
Epoch 00009: val_loss did not improve from 0.85400
307/307 [==============================] - 176s 575ms/step - loss: 0.7849 - acc: 0.6457 - auc: 0.8255 - val_loss: 0.8846 - val_acc: 0.6173 - val_auc: 0.7885
Epoch 10/100
307/307 [==============================] - ETA: 0s - loss: 0.7318 - acc: 0.6750 - auc: 0.8503
Epoch 00010: val_loss did not improve from 0.85400
307/307 [==============================] - 176s 573ms/ste

Epoch 13/100
77/77 [==============================] - ETA: 0s - loss: 0.9043 - acc: 0.5944 - auc_2: 0.7576
Epoch 00013: val_loss improved from 0.91332 to 0.87764, saving model to 3class/1D_CNN_4layers_outputs 1-fold test_setting 100 test data/batch=1024, c1=128, c2=32, c3=64, c4=64, c1filts=15, c2filts=11, c3filts=13, c4filts=3, conv_double=False, globalpool_opt=max, dropout=0.5, dnodes=64, dropout=0/model.hdf5
77/77 [==============================] - 12s 159ms/step - loss: 0.9043 - acc: 0.5944 - auc_2: 0.7576 - val_loss: 0.8776 - val_acc: 0.6211 - val_auc_2: 0.7787
Epoch 14/100
77/77 [==============================] - ETA: 0s - loss: 0.8941 - acc: 0.5986 - auc_2: 0.7635
Epoch 00014: val_loss improved from 0.87764 to 0.87252, saving model to 3class/1D_CNN_4layers_outputs 1-fold test_setting 100 test data/batch=1024, c1=128, c2=32, c3=64, c4=64, c1filts=15, c2filts=11, c3filts=13, c4filts=3, conv_double=False, globalpool_opt=max, dropout=0.5, dnodes=64, dropout=0/model.hdf5
77/77 [=====

...

Epoch 21/100
307/307 [==============================] - ETA: 0s - loss: 0.8575 - acc: 0.6155 - auc: 0.7854
Epoch 00021: val_loss improved from 0.85985 to 0.84816, saving model to 3class/1D_CNN_4layers_outputs 1-fold test_setting 100 test data/batch=256, c1=128, c2=128, c3=128, c4=32, c1filts=15, c2filts=15, c3filts=3, c4filts=3, conv_double=True, globalpool_opt=max, dropout=0.2, dnodes=16, dropout=0.5/model.hdf5
307/307 [==============================] - 45s 146ms/step - loss: 0.8575 - acc: 0.6155 - auc: 0.7854 - val_loss: 0.8482 - val_acc: 0.6463 - val_auc: 0.7835
Epoch 22/100
307/307 [==============================] - ETA: 0s - loss: 0.8544 - acc: 0.6174 - auc: 0.7877
Epoch 00022: val_loss improved from 0.84816 to 0.83074, saving model to 3class/1D_CNN_4layers_outputs 1-fold test_setting 100 test data/batch=256, c1=128, c2=128, c3=128, c4=32, c1filts=15, c2filts=15, c3filts=3, c4filts=3, conv_double=True, globalpool_opt=max, dropout=0.2, dnodes=16, dropout=0.5/model.hdf5
307/307 [===

Epoch 5/100
77/77 [==============================] - ETA: 0s - loss: 0.8568 - acc: 0.6147 - auc: 0.7882
Epoch 00005: val_loss improved from 0.93499 to 0.86413, saving model to 3class/1D_CNN_4layers_outputs 1-fold test_setting 100 test data/batch=1024, c1=64, c2=128, c3=64, c4=128, c1filts=7, c2filts=13, c3filts=13, c4filts=13, conv_double=False, globalpool_opt=ave, dropout=0.4, dnodes=4, dropout=0.3/model.hdf5
77/77 [==============================] - 15s 191ms/step - loss: 0.8568 - acc: 0.6147 - auc: 0.7882 - val_loss: 0.8641 - val_acc: 0.6356 - val_auc: 0.7827
Epoch 6/100
77/77 [==============================] - ETA: 0s - loss: 0.8355 - acc: 0.6248 - auc: 0.7995
Epoch 00006: val_loss did not improve from 0.86413
77/77 [==============================] - 15s 190ms/step - loss: 0.8355 - acc: 0.6248 - auc: 0.7995 - val_loss: 0.9021 - val_acc: 0.6246 - val_auc: 0.7635
Epoch 7/100
77/77 [==============================] - ETA: 0s - loss: 0.8147 - acc: 0.6386 - auc: 0.8107
Epoch 00007: val_lo

Epoch 2/100
154/154 [==============================] - ETA: 0s - loss: 0.9959 - acc: 0.4897 - auc: 0.6863
Epoch 00002: val_loss improved from 1.16164 to 0.98640, saving model to 3class/1D_CNN_4layers_outputs 1-fold test_setting 100 test data/batch=512, c1=256, c2=64, c3=64, c4=128, c1filts=11, c2filts=15, c3filts=13, c4filts=7, conv_double=True, globalpool_opt=max, dropout=0.3, dnodes=0, dropout=0/model.hdf5
154/154 [==============================] - 68s 442ms/step - loss: 0.9959 - acc: 0.4897 - auc: 0.6863 - val_loss: 0.9864 - val_acc: 0.4784 - val_auc: 0.6890
Epoch 3/100
154/154 [==============================] - ETA: 0s - loss: 0.9822 - acc: 0.5096 - auc: 0.7013
Epoch 00003: val_loss improved from 0.98640 to 0.97005, saving model to 3class/1D_CNN_4layers_outputs 1-fold test_setting 100 test data/batch=512, c1=256, c2=64, c3=64, c4=128, c1filts=11, c2filts=15, c3filts=13, c4filts=7, conv_double=True, globalpool_opt=max, dropout=0.3, dnodes=0, dropout=0/model.hdf5
154/154 [===========

Epoch 1/100
154/154 [==============================] - ETA: 0s - loss: 1.0004 - acc: 0.5020 - auc: 0.6900
Epoch 00001: val_loss improved from inf to 0.98523, saving model to 3class/1D_CNN_4layers_outputs 1-fold test_setting 100 test data/batch=512, c1=256, c2=256, c3=512, c4=128, c1filts=13, c2filts=3, c3filts=7, c4filts=11, conv_double=False, globalpool_opt=ave, dropout=0.1, dnodes=8, dropout=0.3/model.hdf5
154/154 [==============================] - 56s 363ms/step - loss: 1.0004 - acc: 0.5020 - auc: 0.6900 - val_loss: 0.9852 - val_acc: 0.5266 - val_auc: 0.7027
Epoch 2/100
154/154 [==============================] - ETA: 0s - loss: 0.9337 - acc: 0.5743 - auc: 0.7421
Epoch 00002: val_loss improved from 0.98523 to 0.96762, saving model to 3class/1D_CNN_4layers_outputs 1-fold test_setting 100 test data/batch=512, c1=256, c2=256, c3=512, c4=128, c1filts=13, c2filts=3, c3filts=7, c4filts=11, conv_double=False, globalpool_opt=ave, dropout=0.1, dnodes=8, dropout=0.3/model.hdf5
154/154 [=======

Epoch 10/100
154/154 [==============================] - ETA: 0s - loss: 0.7990 - acc: 0.6525 - auc: 0.8211
Epoch 00010: val_loss did not improve from 0.90817
154/154 [==============================] - 60s 393ms/step - loss: 0.7990 - acc: 0.6525 - auc: 0.8211 - val_loss: 0.9974 - val_acc: 0.5627 - val_auc: 0.7340
random search 76/200
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Epoch 1/100


## Regression

In [ ]:
from keras import metrics
# random search for hyperparameter
ntrial = 100
train_errs, val_errs = [], []
test_auc, test_rmse, test_acc = [], [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
    test_setting_2 = random.choice(test_settings_2)
    

    # test_setting
    num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4 = test_setting_1
    dense_node, dropout_cnn, dropout_fc, globalpool_opt, batch_size, conv_double = test_setting_2

        

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = 'batch={}, c1={}, c2={}, c3={}, c4={}, c1filts={}, c2filts={}, c3filts={}, c4filts={}, conv_double={}, globalpool_opt={}, dropout={}, dnodes={}, dropout={}'.format(batch_size, num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4, conv_double, globalpool_opt, dropout_cnn, dense_node, dropout_fc)
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/model.hdf5".format(odir)        

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    with strategy.scope():
        # build a model
        model = Sequential()

        conv_act = True
        if conv_act:
            act = 'relu'
        else:
            act = None

        # c1 layer
        if conv_double:
            model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='same'))
        model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))

        # c2 layer
        if conv_double:
            model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='same'))
        model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c3 layer
        if conv_double:
            model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='same'))
        model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c4 layer
        if conv_double:
            model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, padding='same'))
        model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))


        # global이냐 flatten이냐는 따로 모델 나눠야 할듯
        if globalpool_opt == 'max':
            model.add(GlobalMaxPool1D())
        elif globalpool_opt == 'ave':
            model.add(GlobalAveragePooling1D())
            
            
        if dense_node != 0:
            model.add(Dropout(dropout_cnn))
            model.add(Dense(dense_node, activation='sigmoid'))
        model.add(Dropout(dropout_fc))
        model.add(Dense(1, activation='sigmoid'))


        # model 학습 설정
        model.compile(loss='mse', optimizer=Adam(lr=learning_rate), metrics=["mean_absolute_error", tf.keras.metrics.AUC()])
        hist = model.fit(x_train, y_train/10, validation_split=0.1, epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                            EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')])


    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_test).flatten()

    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test_bin, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_auc.append(roc_auc)
    # RMSE 계산
    model_err = metrics.RootMeanSquaredError()
    model_err.update_state(y_test, y_pred)
    rmse_val = model_err.result().numpy()
    test_rmse.append(rmse_val)
    # acc 계산
    acc_val = np.mean((y_pred*10>=5)==y_test_bin)
    test_acc.append(acc_val)
    # rename
    os.rename(odir, rootdir+'/auc{:.4f}_{}_rmse{:.4f}_acc{:.2f}'.format(roc_auc, odir_f, rmse_val, acc_val))

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_auc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_auc(max_idx), random_settings(max_idx)))

# Results

## 1D-CNN model 1
[input]-(conv1-bn-maxpool)-(conv1-bn-maxpool)-(conv1-bn-maxpool)-(conv1-bn-maxpool)-(global maxpool)-dropout-(dense)-dropout-[output]
<br>or conv-conv-bn-maxpool

### Best Result

In [6]:
max_idx = test_auc.index(max(test_auc))
print('\nBest Model roc:{:.4f}\ninfo: {}'.format(test_auc[max_idx], random_settings[max_idx]))


Best Model roc:0.7853
info: batch=256, c1=64, c2=128, c3=32, c4=256, c1filts=11, c2filts=7, c3filts=13, c4filts=5, conv_double=False, globalpool_opt=ave, dropout=0.1, dnodes=0, dropout=0


### Top 10 Result

In [5]:
# top 10 model
topid= sorted(range(len(test_auc)),key= lambda i: test_auc[i])[-10:]

for i in range(10):
    print('Top {} Model: roc {:.4f}   train mse {:.4f}  val mse {:4f}'.format(i+1, np.array(test_auc)[topid[9-i]], np.array(train_errs)[topid[9-i]], np.array(val_errs)[topid[9-i]]))
    print(' {}\n'.format(np.array(random_settings[topid[9-i]])))

    #np.array(test_auc)[topid], np.array(random_settings)[topid]

Top 1 Model: roc 0.7853   train mse 0.2673  val mse 0.482166
 batch=256, c1=64, c2=128, c3=32, c4=256, c1filts=11, c2filts=7, c3filts=13, c4filts=5, conv_double=False, globalpool_opt=ave, dropout=0.1, dnodes=0, dropout=0

Top 2 Model: roc 0.7822   train mse 0.1443  val mse 0.441228
 batch=512, c1=32, c2=32, c3=512, c4=512, c1filts=3, c2filts=5, c3filts=5, c4filts=11, conv_double=False, globalpool_opt=ave, dropout=0.4, dnodes=8, dropout=0.1

Top 3 Model: roc 0.7808   train mse 0.4180  val mse 0.417034
 batch=256, c1=256, c2=512, c3=256, c4=32, c1filts=3, c2filts=3, c3filts=9, c4filts=15, conv_double=True, globalpool_opt=max, dropout=0.5, dnodes=16, dropout=0.2

Top 4 Model: roc 0.7781   train mse 0.4007  val mse 0.428640
 batch=1024, c1=32, c2=256, c3=128, c4=512, c1filts=15, c2filts=15, c3filts=13, c4filts=9, conv_double=True, globalpool_opt=max, dropout=0, dnodes=64, dropout=0.5

Top 5 Model: roc 0.7773   train mse 0.3908  val mse 0.438873
 batch=128, c1=32, c2=32, c3=512, c4=64, c1fi

## 1D-CNN model 2
[input]-(conv1-bn-maxpool)-(conv1-bn-maxpool)-(global max or ave or flatten)-dropout-(dense)-dropout-[output]

In [52]:
max_idx = test_auc.index(max(test_auc))
print('\nBest Model roc:{:.4f}  rmse:{:.4f}  acc:{:.2f}\n: {}'.format(test_auc[max_idx], test_rmse[max_idx], test_acc[max_idx], random_settings[max_idx]))


Best Model roc:0.7496  rmse:4.2166  acc:0.66
: batch=256, c1nodes=64, c2nodes=128, c1filts=11, c2filts=8, globalpool_opt=ave, dropout=0.3, dnodes=8, dropout=0.5


In [53]:
# top 10 model
topid= sorted(range(len(test_auc)),key= lambda i: test_auc[i])[-10:]

for i in range(10):
    print('Top {} Model: roc {:.4f}  rmse:{:.4f}  acc:{:.2f},  train mse {:.4f}  val mse {:4f}'.format(i+1, np.array(test_auc)[topid[9-i]],np.array(test_rmse)[topid[9-i]], np.array(test_acc)[topid[9-i]], np.array(train_errs)[topid[9-i]], np.array(val_errs)[topid[9-i]]))
    print(' {}\n'.format(np.array(random_settings[topid[9-i]])))

    #np.array(test_auc)[topid], np.array(random_settings)[topid]

Top 1 Model: roc 0.7496  rmse:4.2166  acc:0.66,  train mse 0.0415  val mse 0.039510
 batch=256, c1nodes=64, c2nodes=128, c1filts=11, c2filts=8, globalpool_opt=ave, dropout=0.3, dnodes=8, dropout=0.5

Top 2 Model: roc 0.7492  rmse:4.2416  acc:0.59,  train mse 0.0394  val mse 0.040106
 batch=128, c1nodes=64, c2nodes=512, c1filts=3, c2filts=7, globalpool_opt=ave, dropout=0.3, dnodes=32, dropout=0.5

Top 3 Model: roc 0.7426  rmse:4.2331  acc:0.62,  train mse 0.0400  val mse 0.039092
 batch=256, c1nodes=64, c2nodes=64, c1filts=3, c2filts=6, globalpool_opt=max, dropout=0.1, dnodes=32, dropout=0.5

Top 4 Model: roc 0.7426  rmse:4.2217  acc:0.64,  train mse 0.0403  val mse 0.036776
 batch=1024, c1nodes=128, c2nodes=64, c1filts=12, c2filts=10, globalpool_opt=ave, dropout=0.3, dnodes=8, dropout=0.5

Top 5 Model: roc 0.7412  rmse:4.2421  acc:0.60,  train mse 0.0408  val mse 0.044279
 batch=128, c1nodes=64, c2nodes=512, c1filts=10, c2filts=12, globalpool_opt=ave, dropout=0.1, dnodes=16, dropout=0.

In [ ]:
def build_model(num_l1=256, kernel_l1 = 10, bool_flatten=False, dropout=0.2):
    #strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    #with strategy.scope():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    model = Sequential()
    #num_l2=64, num_l3=64, kernel_l2=3, kernel_l3=3
    num_l2=64
    num_l3=64
    kernel_l2=3
    kernel_l3=3

    #for (num_node, kernel_size) in conv_layers:
    #    model.add(Conv1D(filters=num_node, kernel_size=kernel_size, padding='valid'))
    #    model.add(BatchNormalization())
    #    model.add(MaxPooling1D(pool_size=2))

    # Conv Layer 1
    model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='valid'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2)) 

    # Conv Layer 2
    model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='valid'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2)) 

    # Conv Layer 3
    model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='valid'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2)) 

    # FC layer 이전의 작업
    if bool_flatten:
        model.add(Flatten())
    else:
        model.add(GlobalMaxPool1D())

    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


return model